## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database2.csv")
city_data_df.head()

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Weather Description
0,0,Ushuaia,-54.8000,-68.3000,42.46,75,20,13.80,AR,Clouds
1,1,Baghdad,33.3406,44.4009,73.31,46,0,9.22,IQ,Dust
2,2,Khargone,21.8167,75.6000,86.86,44,2,12.95,IN,Clear
3,3,Santo Tomas,14.1079,121.1414,81.55,90,100,3.58,PH,Clouds
4,4,Ilulissat,69.2167,-51.1000,26.62,43,51,4.61,GL,Clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input('Please enter the minimum temperature desired for your vacation:'))
max_temp = float(input('Please enter the maximum temperature desired for your vacation:'))


Please enter the minimum temperature desired for your vacation:60
Please enter the maximum temperature desired for your vacation:95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
suggested_cities = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)]




In [5]:
# 4a. Determine if there are any empty rows.
suggested_cities.count()

City ID                428
City                   428
Lat                    428
Lng                    428
Max Temp               428
Humidity               428
Cloudiness             428
Wind Speed             428
Country                419
Weather Description    428
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows
best_cities_df = suggested_cities.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = best_cities_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Baghdad,IQ,73.31,Dust,33.3406,44.4009,
2,Khargone,IN,86.86,Clear,21.8167,75.6000,
3,Santo Tomas,PH,81.55,Clouds,14.1079,121.1414,
6,Saint-Philippe,RE,74.21,Clear,-21.3585,55.7679,
8,Hithadhoo,MV,85.32,Clouds,-0.6000,73.0833,
12,East London,ZA,61.83,Clear,-33.0153,27.9116,
16,Hermanus,ZA,60.80,Clouds,-34.4187,19.2345,
19,Bure,ET,71.33,Clouds,10.7000,37.0667,
22,Saint George,US,80.26,Clear,37.1041,-113.5841,
23,Castro,BR,61.86,Clouds,-24.7911,-50.0119,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    city_name = row['City']
    params['location'] = f'{lat},{lng}'
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
 

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except(IndexError):
        print(f'Hotel not found near {city_name}...Remove City')

Hotel not found near Jutai...Remove City
Hotel not found near Wattegama...Remove City
Hotel not found near Araouane...Remove City
Hotel not found near Edd...Remove City
Hotel not found near Jurm...Remove City
Hotel not found near Thinadhoo...Remove City
Hotel not found near Adrar...Remove City
Hotel not found near Bukama...Remove City
Hotel not found near Lodja...Remove City
Hotel not found near Micheweni...Remove City
Hotel not found near Murgab...Remove City
Hotel not found near Shurob...Remove City
Hotel not found near Gorom-Gorom...Remove City
Hotel not found near Beinamar...Remove City
Hotel not found near Marzuq...Remove City
Hotel not found near Agadez...Remove City
Hotel not found near Baijiantan...Remove City
Hotel not found near Tessalit...Remove City
Hotel not found near Taoudenni...Remove City
Hotel not found near Sangueya...Remove City
Hotel not found near Madimba...Remove City
Hotel not found near Bangassou...Remove City
Hotel not found near Nioro...Remove City
Hotel not 

In [38]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df['Hotel Name'].replace('', np.nan, inplace = True)
new_hotel_df = hotel_df.dropna()

In [49]:
# 8a. Create the output File (CSV)

output_data_file = 'Weatherpy_vacation.csv'
# 8b. Export the City_Data into a csv
new_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [57]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Weather Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in new_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = new_hotel_df[["Lat", "Lng"]]

In [60]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.25)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))